### Generalized PAP model that get the same format for intensity like benoit's code

To Do:
* Fix X to be calculated directly by matrices product [ ]
* 

In [2]:
import math
import pandas as pd
import numpy as np
import sqlite3
import ZAF as zaf
# plotting
import matplotlib.pyplot as plt
# inputs
elements = ['Fe','Ni']
elements_df = zaf.get_element(elements)
#C = {'Fe':0.1, 'Ni':0.9}
Ec = {'Fe': 7.11, 'Ni': 8.332} ### Check if you have to add all the lines and what to put for H
E = 15 # in keV
take_off_angle = 40
#
excited_element = 'Fe'
excited_line = 'K'
excited_line_mac = 'Ka' 
# import the constants
A = elements_df[['element','molar_mass']]
Z = elements_df[['element','atomic_number']]
density = elements_df[['element','density']]
# import MAC
mac = zaf.get_mac(elements, file_mac='Henke 1993_modified.txt')
print(Z[Z['element'] == 'Fe'])
print(mac)
# Extracting the needed MAC from the Henke database
emission = {'element': excited_element, 'line': excited_line_mac}
mac_df = pd.read_csv("Henke 1993_modified.txt")
sample_mac = pd.DataFrame()
for idx, value in enumerate(elements):
    new_row = pd.DataFrame({'abs': [value], 'mac': [mac_df[(mac_df['zAbs'] == value) & (mac_df['zMes'] == emission['element'])][emission['line']].values[0]] })
    print(new_row)
    sample_mac = sample_mac.append(new_row, ignore_index=True)
print(sample_mac)

## MACKalpha
MACKalpha = mac_df[['zAbs', 'zMes','Ka']]
MACKbeta = mac_df[['zAbs', 'zMes', 'Kb']]
MACLalpha = mac_df[['zAbs', 'zMes', 'La']]
MACLbeta = mac_df[['zAbs', 'zMes', 'Lb']]
MACLl = mac_df[['zAbs', 'zMes', 'Ll']]
MACMa = mac_df[['zAbs', 'zMes', 'Ma']]
MACMb = mac_df[['zAbs', 'zMes', 'Mb']]

# reorganizing the matrix to have 92x92
MACKalpha_np_indices = MACKalpha.to_numpy()
MACKbeta_np_indices = MACKbeta.to_numpy()
MACLalpha_np_indices = MACLalpha.to_numpy()
MACLbeta_np_indices = MACLbeta.to_numpy()
MACLl_np_indices = MACLl.to_numpy()
MACMa_np_indices = MACMa.to_numpy()
MACMb_np_indices = MACMb.to_numpy()

# MAC Matrices
MACKalpha_np = np.zeros((92,92))
MACKbeta_np = np.zeros((92,92))
MACLalpha_np = np.zeros((92,92))
MACLbeta_np = np.zeros((92,92))
MACLl_np = np.zeros((92,92))
MACMa_np = np.zeros((92,92))
MACMb_np = np.zeros((92,92))

for i in range(0,92): # element zAbs
    for j in range(1,93): # element zMes
        MACKalpha_np[i,j-1] = MACKalpha_np_indices[(j-1) + 92 * i,2]
        MACKbeta_np[i,j-1] = MACKbeta_np_indices[(j-1) + 92*i, 2]
        MACLalpha_np[i,j-1] = MACLalpha_np_indices[(j-1) + 92*i, 2]
        MACLbeta_np[i,j-1] = MACLbeta_np_indices[(j-1) + 92*i, 2]
        MACLl_np[i,j-1] = MACLl_np_indices[(j-1) + 92*i, 2]
        MACMa_np[i,j-1] = MACMa_np_indices[(j-1) + 92*i, 2]
        MACMb_np[i,j-1] = MACMb_np_indices[(j-1) + 92*i, 2]

  element  atomic_number
0      Fe             26
     zAbs zMes   Ka   Kb     La     Lb     Ll  Ma  Mb  Mz
2325   Fe   Fe   68   52   2151  17518   2879   0   0 NaN
2327   Fe   Ni  344  267  12403  11986  15342   0   0 NaN
2509   Ni   Fe   94   71   2983   2858   4003   0   0 NaN
2511   Ni   Ni   58   43   1802  13818   2403   0   0 NaN
  abs  mac
0  Fe   68
  abs  mac
0  Ni   94
  abs  mac
0  Fe   68
1  Ni   94


C:\Users\khalil\AppData\Local\Temp\ipykernel_29112\2694053166.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_mac = sample_mac.append(new_row, ignore_index=True)
C:\Users\khalil\AppData\Local\Temp\ipykernel_29112\2694053166.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_mac = sample_mac.append(new_row, ignore_index=True)


In [8]:
# CalcI with numpy matrices like Benoit

# calculate the intensity for one element
def CalcI(C, Eo, mac_I, mac_np, excited_element, excited_line_mac, excited_line, elements, Ec):
    """
         @mac matrix should contain just of the elements present in the sample or standard
    """
    #parameters: E,C,Z,A # output M, J,V, Ro, dE/dps
    elements_df = zaf.get_element(elements)
    A = elements_df[['element','molar_mass']]
    Z = elements_df[['element','atomic_number']]
    # convert the C dictionary into a numpy matrix
    C_numpy = np.zeros((92,1))
    if isinstance(C, np.ndarray) == False:
        for key in C:
            # 1. find the atomic number of that element
            # print('KEY', key)
            # print('Z df', Z)
            index = Z[Z['element'] == key].atomic_number.values - 1 # because in python we start counting from 0
            #print("INDEX ELEMENT", index)
            np.put(C_numpy, [index], [C[key]])
            # print("The element", key)
            # print("The index", index)
        C = C_numpy
    
    # print("C,",C)
    lnJ = 0
    M = 0
    # M - for sample ## SCALAR
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        # print('Z', z_element)
        # print('A', a_element)
        # print('C', C[z_element - 1])
        M += C[z_element - 1] * z_element/a_element
    #print('M', M)
    # Ji, J - for sample
    for element in elements: 
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        Ji = pow(10,-3) * z_element *(10.04 + 8.25 * math.exp(-z_element/11.22)) # in keV
        lnJi = math.log(Ji)
        #print('lnJi, ', lnJi)
        lnJ += (C[z_element - 1]*(z_element/a_element)*lnJi)/M
    J = math.exp(lnJ) # in keV
    #print('[+] J:', J)
    V = Eo/J
    #print('[+] E0:', J)
    # area parameter (number of primary ionizations)
    #1 deceleration of electrons - For sample
    f_V = 0
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    for i in range(3):
        f_V += Dk[i]*(Eo/J)*pow(Eo/J, Pk[i])
    #print('[+] f_V:', f_V)
    # Ro
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Dkrep = np.repeat(np.array(Dk), 92, axis=1)
    print("Dkrep, ", Dkrep.shape)
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Ro = 0
    for i in range(3):
        Ro += (1/M)*(pow(J,1-Pk[i])*Dk[i]*pow(Eo,1+Pk[i])/(1+Pk[i]))
    
    dEdps = -(M/J)*[1/f_V] # keV.g-1.cm2
    #print('[+] R0:', Ro)
    # calculate the deceleration
    
    ##############################################################################################################
    # for the mac database
    # Ionization cross section
    # calculating 1/S
    Uo = Eo/Ec[excited_element]
    
    Vo = V #### IS THIS TRUE???? ####
    m_k=0.86 + 0.12 * math.exp(-pow((Z[Z['element'] == excited_element].atomic_number.values)/5, 2))
    m_values= {'K': m_k,
        'L': 0.82,
        'M': 0.78}
    m = m_values[excited_line]
    T = [1+Pk[0]-m, 1+Pk[1]-m, 1+Pk[2]-m]
    inverse_S = 0
    for i in range(3):
        inverse_S += (Uo/(Vo*M)) * Dk[i]* pow(Vo/Uo, Pk[i]) * (T[i]*pow(Uo, T[i])*math.log(Uo)-pow(Uo,T[i])+1)/pow(T[i],2)
    #print('Inverse_S', inverse_S)
    # print("1/S: ", inverse_S)
    ##############################################################################################################
    # Backscatter factor R
    # After calculating R/S, representing the primary intensity, and Q^A_l(Eo), one determines the area F of the distribution phi(pz):
    # F = (R/S)Q

    ######################## Appendix I
    [}}}]# Z_b_bar
    Z_b_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_b_bar += C[z_element-1] * pow(z_element,1/2)
    Z_b_bar = pow(Z_b_bar, 2)
    # eta bar
    eta_bar = 1.75*pow(10,-3)*Z_b_bar + 0.37*(1-math.exp(-0.015*pow(Z_b_bar, 1.3)))
    #print('eta_bar', eta_bar)
    # W_bar
    W_bar = 0.595 * eta_bar/3.7 + pow(eta_bar,4.55)
    #print('W_bar', W_bar)
    # J(Uo)
    J_Uo = 1+Uo*(math.log(Uo) - 1)
    #print('J_Uo',J_Uo)
    # G(Uo)
    q = (2 * W_bar - 1) / (1 - W_bar)
    G_Uo = (Uo-1-(1-1/pow(Uo, 1+q))/(1+q))/((2+q)*J_Uo)
    R = 1 - eta_bar * W_bar*(1-G_Uo)
    #print('R',R)
    # F
    cross_section = math.log(Uo)/(pow(Uo, m)*Ec[excited_element]*Ec[excited_element])
    #print('Cross section', cross_section)
    F = (R*inverse_S) / cross_section # it was multiplication but corrected by benoit to be division 
    # print("Backscatter factor R:", R)
    # print('CROSS SECTION: ', cross_section)
    # print('Uo:', Uo)
    # print('R :', R)
    # print('1/S: ', inverse_S)
    # print('Q: ',cross_section)
    # print('F:', F)
    ##############################################################################################################
    ######## phi(rhoz)
    r = 2 - 2.3 * eta_bar
    # Phio
    phi0 = 1 + 3.3 * (1-1/pow(Uo,r))* pow(eta_bar, 1.2)
    #print('Phi0',phi0)
    # Z_bar_n
    lnZ_bar_n = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        lnZ_bar_n += C[z_element - 1] * math.log(z_element)
    Z_bar_n = math.exp(lnZ_bar_n)
    #print('Z_bar_n',Z_bar_n)
    # X
    X = pd.DataFrame()
    X_tot = 0
    for idx, value in enumerate(elements):
        # calculating X
        #print(mac_I)
        new_row = pd.DataFrame({'element': [value], 'X': mac_I[(mac_I['zAbs'] == value) & (mac_I['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))})
        X = X.append(new_row, ignore_index=True)    
        # calculating X total
        z_element = Z[Z['element'] == element].atomic_number.values
        print('Element: ', value)
        print('X tot, Z', z_element)
        print('X tot, C', C[z_element-1])
        print('Excited element:', excited_element)
        print('X_tot, mac',mac_I[(mac_I['zAbs'] == value) & (mac_I['zMes'] == excited_element)][excited_line_mac].values[0])
        X_tot = X_tot + C[z_element-1] * mac_I[(mac_I['zAbs'] == value) & (mac_I['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))
    print("MAC", mac_I)
    print("MAC numpy",mac_I.to_numpy())
    #X_tot_numpy = C * mac_I.to_numpy() * (1/math.sin(math.radians(take_off_angle)))
    print('C shape', C.shape)
    print('mac I, shape, ', mac_I.to_numpy().shape)
    X_tot_numpy = np.matmul(mac_np, C) * (1/math.sin(math.radians(take_off_angle)))
    print("X_tot, \n", X_tot_numpy)
    print("X_tot, shape", X_tot_numpy.shape)
    # Phi: parameters pz, phi0, Z_bar_n, M, J) - output: phi, F1, F2, f_X
    D1 = 6.6*pow(10,-6)
    D2 =1.12*pow(10,-5)*(1.35-0.45*pow(J,2))
    D3 = 2.2*pow(10,-6)/J
    Dk = [D1,D2,D3]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Z_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_bar += C[z_element-1] * z_element
    #print('Z_bar',Z_bar)
    # Rx the range of ionization
    # Ro is the total trajectory of the electrons between Eo and El
    # Q is a function of overvolatge and atomic number
    b = 40/Z_bar
    Qo = 1 - 0.535 * math.exp(-pow(21/Z_bar_n, 1/2)) - 2.5*pow(10,-4)*pow(Z_bar_n/20, 3.5) #page 60 why 1/2 instead of 1.2!!!!!!!!!!!!!!!!!!!!!!!!!
    Q = Qo + (1-Qo)*math.exp(-(Uo-1)/b)
    # D
    h = pow(Z_bar, 0.45)
    D = 1+1/pow(Uo,h)
    # Ro
    Ro = 0
    for i in range(3):
        Ro += (1/M) * pow(J, 1-Pk[i]) * Dk[i] * (pow(Eo,1+Pk[i])-pow(Ec[excited_element], 1+Pk[i]))/(1+Pk[i])

    # Rx
    Rx = Q * D * Ro

    # Rm depth of the maximum of the distribution
    #print("Uo-1", Uo-1)
    #if (Uo - 1) > 1:
    G1 = 0.11 * 0.41 * math.exp(-pow(Z_bar/12.75, 0.75))
    G2 = 1 - math.exp(-pow((Uo - 1), 0.35)/1.19)
    G3 = 1 - math.exp(-(Uo-0.5)*pow(Z_bar,0.4)/4)
    Rm = G1 * G2 * G3 * Rx
    # else:
    #     Rm = 0
    #     print('WARNING: ELEMENT NOT EXCITED')
    # d
    d = (Rx-Rm)*((F-(phi0*Rx)/3) * ((Rx-Rm)*F - phi0*Rx*(Rm+Rx/3)))
     # Rc
    Rc = (3/2)
    Rc *= (((F - phi0*Rx/3)/phi0) - (pow(d, 1/2))/(phi0*(Rx-Rm)))

    A1 = phi0 /(Rm*(Rc - Rx * ((Rc/Rm) - 1)))
    B1 = phi0 - A1 * Rm*Rm
    A2 = A1 * (Rc - Rm)/(Rc - Rx)
    # DO we need to calculate this phi?
    # pz = np.linspace(0,1,100)
    # if pz >=0 or pz <= Rc:
    #     phi = A1 * (pz - Rm) * (pz - Rm) + B1
    # if pz > Rc or pz <= Rx:
    #     phi = A2 * (pz - Rx) * (pz - Rx)

    # USING X TOT IS THAT CORRECT?????
    X = X_tot
    # Absorption factor correction
    # F1 is intensity after absorption from 0 to Rc
    # F2 is intensity after absorption from Rc to Rx
    # print('##################################################')
    # print('Rc: ', Rc)
    # print('Rx: ', Rx)
    # print('Rm: ', Rm)
    F1 = (A1/X)*(((Rc-Rm) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (Rc-Rm)*Rx + Rm*(Rc-2/X) + 2/(X*X))
    # Benoit equations
    Int1atRc = math.exp(-X*Rc)*(-B1*(X*X) -  A1*(2 + 2 *X*(Rc - Rm) + (X*X)* (Rc - Rm)*(Rc - Rm)))/(pow(X,3))
    Int1at0 = (-B1*(X*X) -  A1*(2 - 2 * X * Rm + (X*X)* (Rm*Rm)))/(pow(X,3))
    Int1 = Int1atRc - Int1at0

    F2 = (A2/X)
    F2 *= (((Rx-Rc) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (2/(X*X))*math.exp(-X*Rx))
    # Benoit eq
    Int2atRx = -( 2 * A2 * math.exp(-X*Rx) )/(pow(X,3)) #( 2 * A2 .* exp(-Chi.*Rc) )./(Chi.^3);
    Int2atRc = (A2*math.exp(-X*Rc)*(-2-2*X*(Rc - Rx)-(X*X)*(Rc-Rx)*(Rc - Rx)) )/(pow(X,3))
    Int2 = Int2atRx - Int2atRc
    f_X = (F1+F2)/F
    ##############################################################################################################
    # print("[+] dE/dps: ", dEdps)
    # print("[+] V: ", V)
    # print("[+] J: ", J)
    # print("[+] M: ", M)
    # print("[+] Ro [g/cm2]: ", Ro)
    # print('Excited element: ', excited_element)
    # print('Ec: ', Ec[excited_element])
    # print('Z_bar_n: ', Z_bar_n)
    # print('Phi0: ', phi0[0])
    # print('M: ', M[0])
    # print('F1: ', F1[0])
    # print('F1 Benoit', Int1)
    # print('F2: ', F2[0])
    # print('F2 Benoit', Int2)
    # print('f_X: ', f_X[0])

    z_element = Z[Z['element'] == excited_element].atomic_number.values
    intensity = C[z_element-1] * (Int1 + Int2) # C is the concentration of the measured element
    print('Intensity of ', excited_element,' ',intensity[0])
    return intensity
# Defining another function that calculate the intensity of all elements in the sample and return a numpy intensity matrix
### SHOULD MAKE THIS RUN FOR ALL ELEMENTS
elements = ['Fe','Ni']
excited_line_list = ['K','K']
excited_line_mac_list = ['Ka', 'Ka']
C = {'Fe':0.1, 'Ni':0.9}
Ec_dict = {'Fe': 7.11, 'Ni': 8.332} ### Check if you have to add all the lines and what to put for H
take_off_angle = 40
Eo = 15 # keV

def CalcI_sample(elements, excited_line_list, excited_line_mac_list, C, Ec, Eo, mac, mac_np):
    elements_df = zaf.get_element(elements)
    A = elements_df[['element','molar_mass']]
    Z = elements_df[['element','atomic_number']]
    C_numpy = np.zeros((92,1))
    for idx, element in enumerate(elements):
        excited_element = element
        excited_line = excited_line_list[idx]
        excited_line_mac = excited_line_mac_list[idx]
        z_element = Z[Z['element'] == element].atomic_number.values
        C_numpy[z_element-1] = CalcI(C, Eo, mac, mac_np, excited_element, excited_line_mac, excited_line, elements,Ec)
    return C_numpy

print(CalcI_sample(elements, excited_line_list, excited_line_mac_list, C, Ec_dict, Eo, mac, MACKalpha_np))

AxisError: axis 1 is out of bounds for array of dimension 1

In [9]:
## regression algorithm
Isp = np.zeros((92,1)) # (measured) intensity in specimen
Istd = np.zeros((92,1)) # (measured) intensity in standard
Cstd = np.zeros((92,1)) # concentrations in the standard
elements = ['Al','O']
elements_df = zaf.get_element(elements)
A = elements_df[['element','molar_mass']]
Z = elements_df[['element','atomic_number']]
Al_index = Z[Z['element'] == "Al"].atomic_number.values - 1
O_index = Z[Z['element'] == "O"].atomic_number.values - 1

np.put(Isp, [Al_index, O_index], [44700/50/300,44700/50/300]) # Al2O3, conditions: 5kV, 50nA, 300s
np.put(Istd, [Al_index], [93860/50/100]) # Al standard, conditions: 5 kV, 50nA , 100s

np.put(Cstd, [Al_index], [1]) # pure Al sample
# Calculating IntPAPstd in each standard
nStd = 1
#IntPAPstd = np.zeros((92, nStd))
IntPAPstd = np.NaN
std_names = ['Pure Al']
std_elements = ['Al']
std_excited_line_list = ['L']
std_excited_line_mac_list = ['Ll'] # correct this for the quantification of Al is this the L2,3 emission????
std_C = {'Al': 1}
std_Ec = {'Al': 0.0689} # in keV
std_mac = zaf.get_mac(std_elements, file_mac='Henke 1993_modified.txt')
print('[+] Standard intensity calculation ########################### ')
for i in range(nStd):
    print('Standard #',i)
    #print(CalcI_sample(std_elements, std_excited_line_list, std_excited_line_mac_list, std_C, Ec).shape)
    # IntPAPstd[:,i] = CalcI_sample(std_elements, std_excited_line_list, std_excited_line_mac_list, std_C, std_Ec, Eo,mac) # Ec since we index using the name of element
    IntPAPstd = CalcI_sample(std_elements, std_excited_line_list, std_excited_line_mac_list, std_C, std_Ec, Eo,std_mac, MACKalpha_np) # Ec since we index using the name of element
print("Standard '", std_names[0],"' Intensity:", "\n", np.reshape(IntPAPstd, (1,92)))
print('[+] END Standard intensity calculation ########################### ')
# 
# IntPAPstdRshp = IntPAPstd # IntPAPstdRshp = NaN .* Cstd;
IntPAPstdRshp = IntPAPstd
print('Isp,\n', np.reshape(Isp, (1,92)))
print('Istd,\n', np.reshape(Istd, (1,92)))

KRatiosMsrd = (Isp/Istd)
print('KRatiosMsrd,\n', np.reshape(KRatiosMsrd, (1,92)))
# KRatiosMsrdNorm = KRatiosMsrd
# KRatiosMsrdNorm = KRatiosMsrdNorm # to normalize => /repmat(sum(KRatiosMsrdNorm,2),1,92) # normalize
# 

Csp_i = (Isp/Istd) * Cstd # Castaing's approximation for first guess of concentrations
print('Csp_i, \n',np.reshape(Csp_i,(1,92)))
# # Csp_inorm = zeros(size(Csp_i)); Csp_inorm(Csp_i==Csp_i) = Csp_i(Csp_i==Csp_i);
# # Csp_inorm = Csp_inorm./repmat(sum(Csp_inorm,2),1,92);% normalised
# # Csp_i = Csp_inorm
# 

Csp_t = Csp_i
sps_names = ['Al2O3']
sps_elements = ['Al','O']
sps_excited_line_list = ['L','K']
sps_excited_line_mac_list = ['Ll', 'Ka'] # correct this for the quantification of Al is this the L2,3 emission????

sps_Ec = {'Al': 0.0689 , 'O': 0.524} # in keV
sps_mac = zaf.get_mac(sps_elements, file_mac='Henke 1993_modified.txt')

ntests = 1
j = 1
for  i in range(ntests):
    # mdl refer to model
    IntPAPsp_t = CalcI_sample(sps_elements, sps_excited_line_list, sps_excited_line_mac_list, Csp_t, sps_Ec, Eo,sps_mac, MACKalpha_np) # Csp_t 
    IntRatio = IntPAPsp_t/IntPAPstdRshp
    # sumKmdl = sum(IntRatio)
    # sumKmsrd = sum(KRatiosMsrd) # NEED TO BE MODIFIED
    # IntRatioNorm = IntRatio/sumKmdl

    # KRatiosMdlNorm[:,i] = IntRatioNorm #.* Csp_t ./ Cstd(j,:);
    # 
    # rat =  sumKmsrd/sumKmdl
    # 
    # Csp_t = Csp_t * KRatiosMsrd[:,j] #./ IntRatio;%
    # 
    # KRatiosMdl[:,i] = IntRatio * rat #.* Csp_t ./ Cstd(j,:);
    # 
    # Csps[:,i+1] =  Csp_t # KRatiosMsrd(j,:) .* IntPAPstdRshp(j,:) ./ (rat*IntPAPsp_t(i,:)./Csp_t);

[+] Standard intensity calculation ########################### 
Standard # 0


AxisError: axis 1 is out of bounds for array of dimension 1